In [110]:
import numpy as np
from math import *
import os
from os.path import join
import bpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv

In [142]:
scale = 1
subdiv = 1
name = "mesh"
middle_point_cache = {}

In [143]:
def vertex(x, y, z):
    """ Return vertex coordinates fixed to the unit sphere """
    length = np.sqrt(x**2 + y**2 + z**2)
    return [(i * scale) / length for i in (x,y,z)]

In [144]:
def middle_point(point_1, point_2):
    """ Find a middle point and project to the unit sphere """
    smaller_index = min(point_1, point_2)
    greater_index = max(point_1, point_2)
    
    key = "{0}-{1}".format(smaller_index, greater_index)
    
    if key in middle_point_cache:
        return middle_point_cache[key]
    
    # If it's not in cache, then we can cut it
    vert_1 = verts[point_1]
    vert_2 = verts[point_2]
    middle = [sum(i)/2 for i in zip(vert_1, vert_2)]
    
    verts.append(vertex(*middle))
    
    index = len(verts) - 1
    middle_point_cache[key] = index
    
    return index

In [145]:
PHI = (1 + np.sqrt(5)) / 2

In [146]:
verts = [ vertex(-1, PHI, 0), 
         vertex( 1, PHI, 0), 
         vertex(-1, -PHI, 0), 
         vertex( 1, -PHI, 0), 
         vertex(0, -1, PHI), 
         vertex(0, 1, PHI), 
         vertex(0, -1, -PHI), 
         vertex(0, 1, -PHI), 
         vertex( PHI, 0, -1), 
         vertex( PHI, 0, 1), 
         vertex(-PHI, 0, -1), 
         vertex(-PHI, 0, 1), ]

In [147]:
faces = [ 
    # 5 faces around point 0 
    [0, 11, 5], 
    [0, 5, 1], 
    [0, 1, 7], 
    [0, 7, 10], 
    [0, 10, 11], 
    # Adjacent faces 
    [1, 5, 9], 
    [5, 11, 4], 
    [11, 10, 2], 
    [10, 7, 6], 
    [7, 1, 8], 
    # 5 faces around 3 
    [3, 9, 4], 
    [3, 4, 2], 
    [3, 2, 6], 
    [3, 6, 8], 
    [3, 8, 9], 
    # Adjacent faces 
    [4, 9, 5], 
    [2, 4, 11], 
    [6, 2, 10], 
    [8, 6, 7], 
    [9, 8, 1], 
]

In [148]:
for i in range(subdiv):
    faces_subdiv = []
    
    for tri in faces:
        v1 = middle_point(tri[0], tri[1])
        v2 = middle_point(tri[1], tri[2])
        v3 = middle_point(tri[2], tri[0])
        
        faces_subdiv.append([tri[0], v1, v3])
        faces_subdiv.append([tri[1], v2, v1])
        faces_subdiv.append([tri[2], v3, v2])
        faces_subdiv.append([v1, v2, v3])
    
    faces = faces_subdiv

In [149]:
mesh = bpy.data.meshes.new(name)
mesh.from_pydata(verts, [], faces)

obj = bpy.data.objects.new(name, mesh)
bpy.context.collection.objects.link(obj)

bpy.context.view_layer.objects.active = obj
bpy.context.active_object.select_set(state=True)

In [150]:
for face in mesh.polygons:
    face.use_smooth = True

In [151]:
x_axis = np.array([point[0] for point in verts])
y_axis = np.array([point[1] for point in verts])
z_axis = np.array([point[2] for point in verts])

In [152]:
fig = ipv.figure()
scatter = ipv.scatter(x_axis, y_axis, z_axis, marker="sphere")
ipv.show()
ipv.save("plt.html")

In [181]:
def euclidean_distance(p1, p2):
    return np.sqrt(sum(np.square(np.array(list(map(lambda x: x[0] - x[1], zip(p1, p2)))))))

In [195]:
def great_circle_distance(r, p1, p2):
    """ Calculate the great circle distance by vector """
    p1, p2 = np.array(p1), np.array(p2)
    dist = np.arccos(np.matmul(p1, p2)) * r
    return dist

In [196]:
def find_min_dist(verts):
    min_dist = np.inf
    point = verts[0]
    for p in verts[1:]:
        dist = great_circle_distance(scale, point, p)
        min_dist = min(min_dist, dist)
    return min_dist

In [209]:
def find_neibors(verts, min_dist):
    neibors = []
    point = verts[0]
    for p in verts[1:]:
        dist = great_circle_distance(scale, point, p)
        if abs(dist - min_dist) <= 0.001:
            neibors.append(p)
    neibors.append(point)
    return neibors

In [210]:
min_dist = find_min_dist(verts)
neibors = find_neibors(verts, min_dist)
print(len(neibors))

6


In [211]:
xs = np.array([p[0] for p in neibors])
ys = np.array([p[1] for p in neibors])
zs = np.array([p[2] for p in neibors])

In [212]:
fig = ipv.figure()
scatter = ipv.scatter(xs, ys, zs, marker="sphere")
ipv.show()
ipv.save("plt.html")

In [177]:
print(min_dist)

0.5465330578253433


In [ ]:
"icosahedral grid"